# Machine Learning Nanodegree

## Capstone Project: Instacart Market Basket Analysis
### Which products will an Instacart consumer purchase again?



### The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Import Datasets
* [Step 1](#step1): Data Exploration
* [Step 2](#step2): Exploratory Visualizations
* [Step 3](#step3): Preprocessing 
* [Step 4](#step4): Benchmarks
* [Step 5](#step5): Algorithm and Techniques
* [Step 6](#step6): Refinements
* [Step 7](#step7): Algorithm Evaluation and Validation

---
<a id='step0'></a>
## Step 0: Import Datasets

### Import Instacart Data

### Import Dataset


---
<a id='step1'></a>
## Step 1: Data Exploration


<a id='step2'></a>
## Step 2: Exploratory Vizualizations

---
<a id='step3'></a>
## Step 3: Data Preprocessing


### Pre-process the Data


---
<a id='step4'></a>
## Step 4: Benchmarks

---
<a id='step5'></a>
## Step 5: Algorithm and Techniques


---
<a id='step6'></a>
## Step 6: Model Refinements


---
<a id='step7'></a>
## Step 7: Model Evaluation and Validation
